# Deep Neural Network for MNIST Classification

We'll apply all the knowledge from the lectures in this section to write a deep neural network. The problem we've chosen is referred to as the "Hello World" of deep learning because for most students it is the first deep learning algorithm they see.

The dataset is called MNIST and refers to handwritten digit recognition. You can find more about it on Yann LeCun's website (Director of AI Research, Facebook). He is one of the pioneers of what we've been talking about and of more complex approaches that are widely used today, such as covolutional neural networks (CNNs). 

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image). 

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes. 

Our goal would be to build a neural network with 2 hidden layers.

### Importing Packages

In [1]:
# !pip install tensorflow-datasets

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

2025-02-22 17:33:04.701462: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740225784.788376    9121 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740225784.816818    9121 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-22 17:33:05.019388: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Data

In [2]:
# as_supervised splits the data into training and testing as we can use it easily
# with_info gives information about the dataset that we can use 

mnist_data, mnist_info  = tfds.load(name='mnist', with_info=True, as_supervised=True)

I0000 00:00:1740225791.098538    9121 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2784 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5


In [3]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_dir='/home/alexender/tensorflow_datasets/mnist/3.0.1',
    file_format=tfrecord,
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)

In [4]:
# We can split the train and test sets from data 

mnist_train, mnist_test = mnist_data['train'], mnist_data['test']

In [5]:
# We need validation samples so we are spliting training data using the info we have from the dataset and converting it into int64 using TF

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [6]:
def scale(image, label):
    """Now we will generally scale our inputs(eg: from 0 to 1) we will write a function.
    It will take mnist image and its label as input and cast the images as float"""
    image = tf.cast(image, tf.float32)
    
    # To scale our image we have to divide it into 255 because each image consist of values from 0 to 255 in each cell in 28*28 matrix
    
    image = image/255.
    return image, label
    

In [7]:
scaled_train_and_validation_data = mnist_train.map(scale)
scaled_test_data = mnist_test.map(scale)

In [8]:
# Shuffling the data

BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

In [9]:
# Spliting validation set 

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [10]:
# Creating batch for mini batch Gradient Descent 

BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

2025-02-22 17:33:11.483113: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:376] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2025-02-22 17:33:11.775666: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


### Model

#### Outline of the model

In [11]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
                                # tf.keras.Input(shape=(28,28,1)),
                                tf.keras.layers.Flatten(input_shape=(28,28,1)),
                                tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                                tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                                tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                                tf.keras.layers.Dense(output_size, activation='softmax'),
                            ])

/home/alexender/Desktop/Projects/My_projects/envs/Data_Science/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


#### Choosing Optimizer and Loss Function

In [12]:
# This is a classification problem so we are choosing loss function as 'sparse_categorical_crossentropy' as its best for classification 
# For Optimizer we are choosing the best ADAM optimizer 
# custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#### Training 

In [13]:
NUM_EPOCH = 10

model.fit(train_data, epochs=NUM_EPOCH, callbacks=[tf.keras.callbacks.EarlyStopping(patience=2)], verbose=2, validation_data=(validation_inputs, validation_targets))

Epoch 1/10


I0000 00:00:1740225794.644495    9402 service.cc:148] XLA service 0x7bb36c0092e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1740225794.644755    9402 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce GTX 1650 Ti, Compute Capability 7.5
2025-02-22 17:33:14.691823: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1740225794.901733    9402 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1740225796.067588    9402 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


540/540 - 7s - 14ms/step - accuracy: 0.9206 - loss: 0.2678 - val_accuracy: 0.9660 - val_loss: 0.1155
Epoch 2/10
540/540 - 2s - 5ms/step - accuracy: 0.9688 - loss: 0.1009 - val_accuracy: 0.9730 - val_loss: 0.0873
Epoch 3/10
540/540 - 3s - 6ms/step - accuracy: 0.9783 - loss: 0.0696 - val_accuracy: 0.9813 - val_loss: 0.0646
Epoch 4/10
540/540 - 2s - 4ms/step - accuracy: 0.9840 - loss: 0.0519 - val_accuracy: 0.9835 - val_loss: 0.0522
Epoch 5/10
540/540 - 2s - 3ms/step - accuracy: 0.9871 - loss: 0.0402 - val_accuracy: 0.9877 - val_loss: 0.0399
Epoch 6/10
540/540 - 2s - 3ms/step - accuracy: 0.9899 - loss: 0.0312 - val_accuracy: 0.9863 - val_loss: 0.0427
Epoch 7/10
540/540 - 2s - 3ms/step - accuracy: 0.9900 - loss: 0.0305 - val_accuracy: 0.9893 - val_loss: 0.0326
Epoch 8/10
540/540 - 2s - 3ms/step - accuracy: 0.9918 - loss: 0.0253 - val_accuracy: 0.9887 - val_loss: 0.0377
Epoch 9/10
540/540 - 1s - 3ms/step - accuracy: 0.9931 - loss: 0.0215 - val_accuracy: 0.9927 - val_loss: 0.0215
Epoch 10/10

### Testing the model 

In [14]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 890ms/step - accuracy: 0.9798 - loss: 0.0871


In [15]:
print("Test Loss: {:.2f}, Test Accuracy: {:.2f}".format(test_loss, test_accuracy*100))

Test Loss: 0.09, Test Accuracy: 97.98


In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 200)            │       157,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 200)            │        40,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 200)            │        40,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         2,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 718,232 (2.74 MB)

 Trainable params: 239,410 (935.20 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 478,822 (1.83 MB)